In [4]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your service account JSON key file
SERVICE_ACCOUNT_FILE = 'creds/google__sa.json'

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate and create the service
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('drive', 'v3', credentials=credentials)

# Replace with your Google Drive folder ID
folder_id = '1sfPFHkXYpKyY41V0pfz3Qw3k4VLy5Hvb'

# Query to get files from the folder
query = f"'{folder_id}' in parents"

# Specify the page size
page_size = 1000  # You can change this number as needed

# Initialize page token
page_token = None


all_files = []
# Loop through pages of results
while True:
    response = service.files().list(q=query,
                                    spaces='drive',
                                    fields='nextPageToken, files(id, name, webViewLink)',
                                    pageSize=page_size,
                                    pageToken=page_token).execute()

    # Print file names and URLs
    for file in response.get('files', []):
        all_files.append(file)
        print('Found file: %s (%s) %s' % (file.get('name'), file.get('id'), file.get('webViewLink')))

    # Update the page token
    page_token = response.get('nextPageToken', None)
    if page_token is None:
        break


Found file: web_services__query__3.ipynb (1cqQW3bV0nWZFuz-eYlfLxZ4gG_1kTDap) https://drive.google.com/file/d/1cqQW3bV0nWZFuz-eYlfLxZ4gG_1kTDap/view?usp=drivesdk
Found file: web_services__query__4.ipynb (1nJqAbuEAm7gGoJvAS6iFNjHWnWOqHpyk) https://drive.google.com/file/d/1nJqAbuEAm7gGoJvAS6iFNjHWnWOqHpyk/view?usp=drivesdk
Found file: web_services__query__2.ipynb (1vzSoYA94hTkADvFI_3MvTD1XNcUagijb) https://drive.google.com/file/d/1vzSoYA94hTkADvFI_3MvTD1XNcUagijb/view?usp=drivesdk
Found file: web_services__query__1.ipynb (1ThC4J8IGoXoTge9HWfouXUANvHAtIGWM) https://drive.google.com/file/d/1ThC4J8IGoXoTge9HWfouXUANvHAtIGWM/view?usp=drivesdk
Found file: web_services__query__0.ipynb (1N6Mdh_sAb2-pTRETZ8-fi36yjEbFZpYG) https://drive.google.com/file/d/1N6Mdh_sAb2-pTRETZ8-fi36yjEbFZpYG/view?usp=drivesdk
Found file: web_services__modification__2.ipynb (1m2wMEyo8zMgStvkZ4zUSrjmXdemR9vfq) https://drive.google.com/file/d/1m2wMEyo8zMgStvkZ4zUSrjmXdemR9vfq/view?usp=drivesdk
Found file: web_services__m

In [5]:
all_files[:10]

[{'webViewLink': 'https://drive.google.com/file/d/1cqQW3bV0nWZFuz-eYlfLxZ4gG_1kTDap/view?usp=drivesdk',
  'id': '1cqQW3bV0nWZFuz-eYlfLxZ4gG_1kTDap',
  'name': 'web_services__query__3.ipynb'},
 {'webViewLink': 'https://drive.google.com/file/d/1nJqAbuEAm7gGoJvAS6iFNjHWnWOqHpyk/view?usp=drivesdk',
  'id': '1nJqAbuEAm7gGoJvAS6iFNjHWnWOqHpyk',
  'name': 'web_services__query__4.ipynb'},
 {'webViewLink': 'https://drive.google.com/file/d/1vzSoYA94hTkADvFI_3MvTD1XNcUagijb/view?usp=drivesdk',
  'id': '1vzSoYA94hTkADvFI_3MvTD1XNcUagijb',
  'name': 'web_services__query__2.ipynb'},
 {'webViewLink': 'https://drive.google.com/file/d/1ThC4J8IGoXoTge9HWfouXUANvHAtIGWM/view?usp=drivesdk',
  'id': '1ThC4J8IGoXoTge9HWfouXUANvHAtIGWM',
  'name': 'web_services__query__1.ipynb'},
 {'webViewLink': 'https://drive.google.com/file/d/1N6Mdh_sAb2-pTRETZ8-fi36yjEbFZpYG/view?usp=drivesdk',
  'id': '1N6Mdh_sAb2-pTRETZ8-fi36yjEbFZpYG',
  'name': 'web_services__query__0.ipynb'},
 {'webViewLink': 'https://drive.google.c

In [7]:
folder_id = all_files[0]['id']
colab_url = f"https://colab.research.google.com/drive/{folder_id}"

print("Colab URL:", colab_url)
print("Drive URL:", all_files[0]['webViewLink'])

Colab URL: https://colab.research.google.com/drive/1cqQW3bV0nWZFuz-eYlfLxZ4gG_1kTDap
Drive URL: https://drive.google.com/file/d/1cqQW3bV0nWZFuz-eYlfLxZ4gG_1kTDap/view?usp=drivesdk


In [12]:
all_colab_urls = []
for file in all_files:
    folder_id = file['id']
    colab_url = f"https://colab.research.google.com/drive/{folder_id}"
    all_colab_urls.append(colab_url)

In [24]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import nbformat

def download_and_extract_metadata(service_account_file, file_id):
    # Authenticate with the service account
    credentials = service_account.Credentials.from_service_account_file(
        service_account_file, scopes=['https://www.googleapis.com/auth/drive'])
    service = build('drive', 'v3', credentials=credentials)

    # Request to download the file
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    # Download the file
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print("Download progress: %d%%." % int(status.progress() * 100))

    # Move the buffer's pointer to the beginning
    fh.seek(0)

    # Open the notebook
    notebook = nbformat.read(fh, as_version=4)

    # Extract the first cell
    first_cell = notebook.cells[0]

    lines = first_cell["source"].split("\n")
    metadata = {}
    for line in lines:
        if "**Python Topics**" in line:
            metadata["topic"] = line.split(" - ")[1]
        if "**Type**" in line:
            metadata["type"] = line.split(" - ")[1]
        if "**Target Number of Turns (User + Assistant)**" in line:
            metadata["target_turns"] = line.split(" - ")[1]

    return metadata

# Usage
service_account_file = 'creds/google__sa.json'
file_id = '1nJqAbuEAm7gGoJvAS6iFNjHWnWOqHpyk'
metadata = download_and_extract_metadata(service_account_file, file_id)
print(metadata)

Download progress: 100%.
{'topic': 'web_development > web_services', 'type': 'query', 'target_turns': '2+'}


In [15]:
# Divide the URLs into batches of 335 (Randomly)
import random
random.shuffle(all_colab_urls)
batch_3 = all_colab_urls[:335]
batch_4 = all_colab_urls[335:670]
batch_5 = all_colab_urls[670:]
batches = [batch_3, batch_4, batch_5]

In [18]:
shared_already = [
    "https://colab.research.google.com/drive/1lHYB-8JiU67LlaqjvaRuLYUbetxWbnD5",
    "https://colab.research.google.com/drive/1cqQW3bV0nWZFuz-eYlfLxZ4gG_1kTDap",
    "https://colab.research.google.com/drive/1nJqAbuEAm7gGoJvAS6iFNjHWnWOqHpyk"
]

for batch in batches:
    for url in batch:
        if url in shared_already:
            # batch.pop(batch.index(url))
            print(url)

In [25]:
import threading

def process_file(url, service_account_file, data_batch):
    file_id = url.split("/")[-1]
    metadata = download_and_extract_metadata(service_account_file, file_id)
    data = {
        "task_link": url,
        "assigned_to_email": "",
        "completion_status": "Unclaimed",
        "modified_question": "",
        "duration_mins": "",
        "completion_date": "",
        "comments": "",
        "metadata__type": metadata["type"],
        "metadata__topic": metadata["topic"],
        "metadata__target_length": metadata["target_turns"],
    }
    data_batch.append(data)

# List to hold thread objects
threads = []

# List to hold processed data
data_batch_3 = []

for url in batch_3:
    # Create a new thread for each file
    thread = threading.Thread(target=process_file, args=(url, service_account_file, data_batch_3))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

# At this point, data_batch_3 contains all the processed data
data_batch_3[:10]

Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.


[{'task_link': 'https://colab.research.google.com/drive/13CZ3MABCvLktnGa6IGADVZEyreu50eZE',
  'assigned_to_email': '',
  'completion_status': 'Unclaimed',
  'modified_question': '',
  'duration_mins': '',
  'completion_date': '',
  'comments': '',
  'metadata__type': 'query',
  'metadata__topic': 'python_language_and_scripting > metaclasses_and_class_factories',
  'metadata__target_length': '2+'},
 {'task_link': 'https://colab.research.google.com/drive/1RwxfUiKbWKsrw202cEs-UdqihvGx47xE',
  'assigned_to_email': '',
  'completion_status': 'Unclaimed',
  'modified_question': '',
  'duration_mins': '',
  'completion_date': '',
  'comments': '',
  'metadata__type': 'query',
  'metadata__topic': 'web_development > web_scraping',
  'metadata__target_length': '2+'},
 {'task_link': 'https://colab.research.google.com/drive/13OnWsmVuXCon_Sk4DiX0_kgEjjHkPXRc',
  'assigned_to_email': '',
  'completion_status': 'Unclaimed',
  'modified_question': '',
  'duration_mins': '',
  'completion_date': '',
 

In [27]:
import pandas as pd

df = pd.DataFrame(data_batch_3)
df

,task_link,assigned_to_email,completion_status,modified_question,duration_mins,completion_date,comments,metadata__type,metadata__topic,metadata__target_length
0,https://colab.research.google.com/drive/13CZ3M...,,Unclaimed,,,,,query,python_language_and_scripting > metaclasses_an...,2+
1,https://colab.research.google.com/drive/1RwxfU...,,Unclaimed,,,,,query,web_development > web_scraping,2+
2,https://colab.research.google.com/drive/13OnWs...,,Unclaimed,,,,,query,data_analysis > json_parsing,2+
3,https://colab.research.google.com/drive/1crm5v...,,Unclaimed,,,,,query,algorithms > by_topic > game_theory,2+
4,https://colab.research.google.com/drive/1WSeNR...,,Unclaimed,,,,,query,web_development > web_design,2+
...,...,...,...,...,...,...,...,...,...,...
328,https://colab.research.google.com/drive/1AEjaY...,,Unclaimed,,,,,query,deep_learning > recurrent_neural_networks,2+
329,https://colab.research.google.com/drive/1Om0cX...,,Unclaimed,,,,,query,algorithms > by_topic > game_theory,2+
330,https://colab.research.google.com/drive/1MTYGH...,,Unclaimed,,,,,modification,algorithms > by_data_structure > hash_tables,2+
331,https://colab.research.google.com/drive/19p4Jl...,,Unclaimed,,,,,query,unit_testing_methodology > unit_testing_basics,2+


In [22]:
df.to_csv("batch_3.csv", index=False)

In [30]:
df_3 = pd.read_csv("batch_3.csv")
df_3

,0
0,https://colab.research.google.com/drive/1T_Jkg...
1,https://colab.research.google.com/drive/13OnWs...
2,https://colab.research.google.com/drive/1xTwo8...
3,https://colab.research.google.com/drive/1s6QrF...
4,https://colab.research.google.com/drive/1uwo3Y...
...,...
328,https://colab.research.google.com/drive/1aNfPg...
329,https://colab.research.google.com/drive/1QhKUP...
330,https://colab.research.google.com/drive/1yd4hr...
331,https://colab.research.google.com/drive/1CQSKk...


In [35]:
df_3.merge(df, left_on="0", right_on="task_link", how="inner").to_csv("batch_3_all_data.csv", index=False)